In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [5]:
!unzip /content/drive/MyDrive/training.zip
!unzip /content/drive/MyDrive/test.zip

Streaming output truncated to the last 5000 lines.
  inflating: training/background/door_38_rotate_none.jpg  
  inflating: training/background/door_38_rotate_none_d2.jpg  
  inflating: training/background/door_38_rotate_none_d3.jpg  
  inflating: training/background/door_38_rotate_sharpen.jpg  
  inflating: training/background/door_38_rotate_sharpen_d2.jpg  
  inflating: training/background/door_38_rotate_sharpen_d3.jpg  
  inflating: training/background/door_39_flip_blur.jpg  
  inflating: training/background/door_39_flip_blur_d2.jpg  
  inflating: training/background/door_39_flip_blur_d3.jpg  
  inflating: training/background/door_39_flip_edgeEnhance.jpg  
  inflating: training/background/door_39_flip_edgeEnhance_d2.jpg  
  inflating: training/background/door_39_flip_edgeEnhance_d3.jpg  
  inflating: training/background/door_39_flip_none.jpg  
  inflating: training/background/door_39_flip_none_d2.jpg  
  inflating: training/background/door_39_flip_none_d3.jpg  
  inflating: training/

#**Process data and save in archive**

In [6]:
def read_img(img):
    img=tf.io.read_file(img)
    img=tf.image.decode_jpeg(img,channels=1)
    return img
def display(img):
    plt.imshow(np.squeeze(img),cmap='gray')
    plt.show()
def load_data():
    # human assumption of test image classes
    test_non_text=[10,12,14,16,18,2,31,32,33,34,
                  35,36,37,38,39,4,41,43,45,47,
                  49,56,57,58,59,6,60,64,65,66,74,
                  75,76,77,78,79,8,80,85,86,87,88,89,
                  95,96,97,98]
    
    x_test=[]
    y_test=[]
    for i in range(1,99):
        path='/content/test/'+str(i)+'.jpg'
        data=read_img(path)
        x_test.append(data)
        if i in test_non_text:
            y_test.append(0)
        else:
            y_test.append(1)
    x_train=[]
    y_train=[]
    import glob
    for path in glob.glob("/content/training/hi/*.jpg"):
        data=read_img(path)
        x_train.append(data)
        y_train.append(1)
    for path in glob.glob("/content/training/background/*.jpg"):
        data=read_img(path)
        x_train.append(data)
        y_train.append(0)
    
    return (x_train,y_train),(x_test,y_test)
    

(x_train,y_train),(x_test,y_test)=load_data()
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)       
    
np.savez('data.npz',x_train,y_train,x_test,y_test)

#**Extract data from the archive**

In [7]:
data=np.load('data.npz')
x_train=data['arr_0']
y_train=data['arr_1']
x_test=data['arr_2']
y_test=data['arr_3']

#**Neural Net architecture**

In [8]:
def CON():
        input=tf.keras.Input(shape=(64,64,1))
        conv1=tf.keras.layers.Conv2D(filters=32,kernel_size=5,
                                    activation='relu',padding='same')
        
        maxpool1=tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid')

        conv2=tf.keras.layers.Conv2D(filters=64,kernel_size=5,
                                    activation='relu',padding='valid')
        
        maxpool2=tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid')
        conv3=tf.keras.layers.Conv2D(filters=64,kernel_size=5,
                                    activation='relu',padding='valid')
        
        maxpool3=tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid')
        
        flat=tf.keras.layers.Flatten()
        fc1=tf.keras.layers.Dense(units=240,activation='relu')
        fc2=tf.keras.layers.Dense(units=128,activation='relu')
        final=tf.keras.layers.Dense(units=1,activation='sigmoid')
        layers=[input,conv1,maxpool1,conv2,maxpool2,conv3,maxpool3,flat,fc1,fc2,final]
        model=tf.keras.Sequential(layers)
        model.compile(loss='BinaryCrossentropy',optimizer='Adam',metrics=['accuracy'])

        return model

cnn=CON()
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=True,
)
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

#**Neural Net training**

In [13]:
cnn.fit(x_train/255.0,y_train,epochs=100,validation_data=(x_test/255.0,y_test),callbacks=callback)

Epoch 1/100
184/184 [==============================] - 3s 14ms/step - loss: 0.0930 - accuracy: 0.9733 - val_loss: 0.2409 - val_accuracy: 0.9796
Epoch 2/100
184/184 [==============================] - 2s 14ms/step - loss: 0.0374 - accuracy: 0.9886 - val_loss: 0.0218 - val_accuracy: 0.9898
Epoch 3/100
184/184 [==============================] - 3s 14ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.0620 - val_accuracy: 0.9796
Epoch 4/100
184/184 [==============================] - 2s 13ms/step - loss: 0.0195 - accuracy: 0.9944 - val_loss: 0.0171 - val_accuracy: 0.9898
Epoch 5/100
184/184 [==============================] - 3s 14ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0125 - val_accuracy: 0.9898
Epoch 6/100
184/184 [==============================] - 2s 14ms/step - loss: 0.0199 - accuracy: 0.9949 - val_loss: 0.0091 - val_accuracy: 0.9898
Epoch 7/100
184/184 [==============================] - 2s 14ms/step - loss: 0.0062 - accuracy: 0.9974 - val_loss: 0.1840 - val_accuracy:

#**Save model**

In [14]:
cnn.save('CON')

INFO:tensorflow:Assets written to: CON/assets


#**Model loading and evaluation on test data**

In [15]:
CON=tf.keras.models.load_model('CON')
CON.evaluate(x_test/255,y_test)

4/4 [==============================] - 0s 7ms/step - loss: 0.0071 - accuracy: 1.0000


[0.007068065460771322, 1.0]

#**Creation of submission json file**

In [16]:
import random
import json
# from utils.io import write_json
def read_img(img):
    img=tf.io.read_file(img)
    img=tf.image.decode_jpeg(img,channels=1)
    return img
def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in range(1,99):
        image_no = str(i) + '.jpg'
        image=read_img('test/'+image_no)
        result=CON.predict(tf.expand_dims(image/255,axis=0))
        if result>0.5:
            res[image_no]=1
        else:
            res[image_no]=0
    write_json(filename, res)

if __name__ == '__main__':
    generate_sample_file('./submission.json')